In [1]:
import asyncio
from contextlib import AsyncExitStack
from typing import Union, Literal


from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from pydantic import BaseModel, create_model, Field

In [2]:
server_params = StdioServerParameters(
    command="python", # Executable
    args=["test_mcp_server.py"], # Optional command line arguments
    env=None # Optional environment variables
)

In [3]:
context_stack = AsyncExitStack()

In [ ]:
read, write = await context_stack.enter_async_context(stdio_client(server_params))
session = await context_stack.enter_async_context(ClientSession(read, write))

cur_loop = asyncio.get_event_loop()
cur_loop.create_task(session.initialize())



<Task pending name='Task-7' coro=<ClientSession.initialize() running at /workspaces/pyllments/.venv/lib/python3.13/site-packages/mcp/client/session.py:76>>

In [5]:
tools =await session.list_tools()

In [9]:
tools.tools[0].model_dump()

{'name': 'calculate',
 'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
 'inputSchema': {'properties': {'operation': {'title': 'Operation',
    'type': 'string'},
   'a': {'title': 'A', 'type': 'number'},
   'b': {'title': 'B', 'type': 'number'}},
  'required': ['operation', 'a', 'b'],
  'title': 'calculateArguments',
  'type': 'object'}}

In [12]:
tc1 = await session.call_tool('calculate', arguments={'a': 1, 'b': 2, 'operation': 'add'})

In [14]:
tc1.model_dump()

{'meta': None,
 'content': [{'type': 'text',
   'text': 'Result of 1.0 add 2.0 = 3.0',
   'annotations': None}],
 'isError': False}

In [16]:
tc1fail =  await session.call_tool('calculate', arguments={'a': 1, 'b': 2, 'operation': 'dddd'})
tc1fail.model_dump()

{'meta': None,
 'content': [{'type': 'text',
   'text': "Error: Unknown operation 'dddd'. Use add, subtract, multiply, or divide.",
   'annotations': None}],
 'isError': False}

In [17]:
tc2 = await session.call_tool('generate_random_number', arguments={'min_value': 1})
tc2.model_dump()


{'meta': None,
 'content': [{'type': 'text', 'text': '70', 'annotations': None}],
 'isError': False}

In [18]:
print(tc2.model_dump()['content'])

[{'type': 'text', 'text': '70', 'annotations': None}]


In [6]:
tools.model_dump()

{'meta': None,
 'nextCursor': None,
 'tools': [{'name': 'calculate',
   'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
   'inputSchema': {'properties': {'operation': {'title': 'Operation',
      'type': 'string'},
     'a': {'title': 'A', 'type': 'number'},
     'b': {'title': 'B', 'type': 'number'}},
    'required': ['operation', 'a', 'b'],
    'title': 'calculateArguments',
    'type': 'object'}},
  {'name': 'get_current_time',
   'description': 'Get the current date and time.',
   'inputSchema': {'properties': {},
    'title': 'get_current_timeArguments',
    'type': 'object'}},
  {'name': 'generate_random_number',
   'description': '\nGenerate a random number between min_value and max_value (inclusive).\n\nArgs:\n    min_value: Minimum value (default: 1)\n    max_value: Maximum value (defa

In [29]:
tool.model_dump()

{'name': 'calculate',
 'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
 'inputSchema': {'properties': {'operation': {'title': 'Operation',
    'type': 'string'},
   'a': {'title': 'A', 'type': 'number'},
   'b': {'title': 'B', 'type': 'number'}},
  'required': ['operation', 'a', 'b'],
  'title': 'calculateArguments',
  'type': 'object'}}

In [31]:
import panel as pn
pn.extension()

In [33]:
pn.pane.JSON(tools.model_dump())

BokehModel(combine_events=True, render_bundle={'docs_json': {'181f398c-1430-4705-a382-088001268591': {'version…

In [41]:
import json

In [47]:
rs = resources.model_dump()

In [49]:
rs

{'meta': None,
 'nextCursor': None,
 'resources': [{'uri': AnyUrl('weather://current'),
   'name': 'weather://current',
   'description': None,
   'mimeType': 'text/plain',
   'size': None,
   'annotations': None},
  {'uri': AnyUrl('help://commands'),
   'name': 'help://commands',
   'description': None,
   'mimeType': 'text/plain',
   'size': None,
   'annotations': None}]}

In [51]:
await session.read_resource('weather://current')

ReadResourceResult(meta=None, contents=[TextResourceContents(uri=AnyUrl('weather://current'), mimeType='text/plain', text='Current weather: Snowy, 15°C')])

In [45]:
type(resources.model_dump())

dict

In [40]:
pn.pane.JSON(resources.model_dump())

ValueError: JSON pane does not support objects of type 'dict'.

In [36]:
resources = await session.list_resources()
pn.pane.JSON(resources.model_dump())





ValueError: JSON pane does not support objects of type 'dict'.

# Truthiness testing in chaining classes

In [61]:
class A:
    def __init__(self, val: list):
        self.val = val
    def __gt__(self, other):
        self.val.extend(other.val)
        return self

In [62]:
a = A(val=[1,2,3])
b = A(val=[4,5,6])
c = A(val=[7,8,9])
a > b > c
a.val, b.val, c.val


([1, 2, 3, 4, 5, 6], [4, 5, 6, 7, 8, 9], [7, 8, 9])

In [57]:
a

In [13]:
import jinja2

In [17]:
from jinja2 import Template
import json

def get_str():
    """Return a Jinja template string for rendering."""
    return """{
    "message": "{{ content }}"
}"""  # Jinja template string with a placeholder for content

json.loads(Template(get_str()).render(content="blah blah blah"))

{'message': 'blah blah blah'}

In [1]:
from pyllments.payloads import ToolResponsePayload

# Pydantic

In [8]:
from pydantic import BaseModel

In [12]:
tools.tools[0].model_dump()

{'name': 'calculate',
 'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
 'inputSchema': {'properties': {'operation': {'title': 'Operation',
    'type': 'string'},
   'a': {'title': 'A', 'type': 'number'},
   'b': {'title': 'B', 'type': 'number'}},
  'required': ['operation', 'a', 'b'],
  'title': 'calculateArguments',
  'type': 'object'}}

In [30]:
from mcp import Tool
from typing import Union

In [56]:
tool = tools.tools[0]


AttributeError: 'Tool' object has no attribute 'parameters'

In [57]:
tool.model_config

{'extra': 'allow'}

In [36]:
class ToolResponse(BaseModel):
    tools: list[Union[*tools.tools]]

In [40]:
ToolResponse.model_json_schema()

{'$defs': {'Tool': {'additionalProperties': True,
   'description': 'Definition for a tool the client can call.',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Description'},
    'inputSchema': {'title': 'Inputschema', 'type': 'object'}},
   'required': ['name', 'inputSchema'],
   'title': 'Tool',
   'type': 'object'}},
 'properties': {'tools': {'items': {'$ref': '#/$defs/Tool'},
   'title': 'Tools',
   'type': 'array'}},
 'required': ['tools'],
 'title': 'ToolResponse',
 'type': 'object'}

In [29]:
ToolResponse(tools=tools.tools).model_dump()

{'tools': [{'name': 'calculate',
   'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
   'inputSchema': {'properties': {'operation': {'title': 'Operation',
      'type': 'string'},
     'a': {'title': 'A', 'type': 'number'},
     'b': {'title': 'B', 'type': 'number'}},
    'required': ['operation', 'a', 'b'],
    'title': 'calculateArguments',
    'type': 'object'}},
  {'name': 'get_current_time',
   'description': 'Get the current date and time.',
   'inputSchema': {'properties': {},
    'title': 'get_current_timeArguments',
    'type': 'object'}},
  {'name': 'generate_random_number',
   'description': '\nGenerate a random number between min_value and max_value (inclusive).\n\nArgs:\n    min_value: Minimum value (default: 1)\n    max_value: Maximum value (default: 100)\n    \nReturns:\n    A ran

In [61]:
tools.model_dump()

{'meta': None,
 'nextCursor': None,
 'tools': [{'name': 'calculate',
   'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
   'inputSchema': {'properties': {'operation': {'title': 'Operation',
      'type': 'string'},
     'a': {'title': 'A', 'type': 'number'},
     'b': {'title': 'B', 'type': 'number'}},
    'required': ['operation', 'a', 'b'],
    'title': 'calculateArguments',
    'type': 'object'}},
  {'name': 'get_current_time',
   'description': 'Get the current date and time.',
   'inputSchema': {'properties': {},
    'title': 'get_current_timeArguments',
    'type': 'object'}},
  {'name': 'generate_random_number',
   'description': '\nGenerate a random number between min_value and max_value (inclusive).\n\nArgs:\n    min_value: Minimum value (default: 1)\n    max_value: Maximum value (defa

{'name': 'calculate',
 'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
 'inputSchema': {'properties': {'operation': {'title': 'Operation',
    'type': 'string'},
   'a': {'title': 'A', 'type': 'number'},
   'b': {'title': 'B', 'type': 'number'}},
  'required': ['operation', 'a', 'b'],
  'title': 'calculateArguments',
  'type': 'object'}}

In [ ]:
tools[]

# Make a union of the specific tool specs

In [7]:
tools.model_dump()

{'meta': None,
 'nextCursor': None,
 'tools': [{'name': 'calculate',
   'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
   'inputSchema': {'properties': {'operation': {'title': 'Operation',
      'type': 'string'},
     'a': {'title': 'A', 'type': 'number'},
     'b': {'title': 'B', 'type': 'number'}},
    'required': ['operation', 'a', 'b'],
    'title': 'calculateArguments',
    'type': 'object'}},
  {'name': 'get_current_time',
   'description': 'Get the current date and time.',
   'inputSchema': {'properties': {},
    'title': 'get_current_timeArguments',
    'type': 'object'}},
  {'name': 'generate_random_number',
   'description': '\nGenerate a random number between min_value and max_value (inclusive).\n\nArgs:\n    min_value: Minimum value (default: 1)\n    max_value: Maximum value (defa

In [13]:
tools.tools

[Tool(name='calculate', description="\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n", inputSchema={'properties': {'operation': {'title': 'Operation', 'type': 'string'}, 'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['operation', 'a', 'b'], 'title': 'calculateArguments', 'type': 'object'}),
 Tool(name='get_current_time', description='Get the current date and time.', inputSchema={'properties': {}, 'title': 'get_current_timeArguments', 'type': 'object'}),
 Tool(name='generate_random_number', description='\nGenerate a random number between min_value and max_value (inclusive).\n\nArgs:\n    min_value: Minimum value (default: 1)\n    max_value: Maximum value (default: 100)\n    \nReturns:\n    A random integer\n', inputSchema={'properties': {'min_value': {'default': 1, 'ti

In [11]:
tools.tools[0].model_dump()


{'name': 'calculate',
 'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
 'inputSchema': {'properties': {'operation': {'title': 'Operation',
    'type': 'string'},
   'a': {'title': 'A', 'type': 'number'},
   'b': {'title': 'B', 'type': 'number'}},
  'required': ['operation', 'a', 'b'],
  'title': 'calculateArguments',
  'type': 'object'}}

In [15]:
from typing import Union
from pydantic import BaseModel

class ToolList(BaseModel):
    tools: list[Union[*tools.tools]]

ToolList.model_json_schema()





{'$defs': {'Tool': {'additionalProperties': True,
   'description': 'Definition for a tool the client can call.',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'description': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default': None,
     'title': 'Description'},
    'inputSchema': {'title': 'Inputschema', 'type': 'object'}},
   'required': ['name', 'inputSchema'],
   'title': 'Tool',
   'type': 'object'}},
 'properties': {'tools': {'items': {'$ref': '#/$defs/Tool'},
   'title': 'Tools',
   'type': 'array'}},
 'required': ['tools'],
 'title': 'ToolList',
 'type': 'object'}

In [36]:
from typing import Union, Literal
from pydantic import BaseModel, create_model, Field

class Tool(BaseModel):
    """A model representing a tool with a name."""
    name: str

# Create specific tool models with predefined names
tool1 = create_model(
    'Tool1',
    name=(Literal['tool1'], ...),
    parameters=(object, Field(json_schema_extra=tools.tools[0].inputSchema))
    )
tool2 = create_model(
    'Tool2',
    name=(Literal['tool2'], ...),
    parameters=(object, Field(json_schema_extra=tools.tools[1].inputSchema))
    )

# Create a model for a list of tools that can be either tool1 or tool2
ToolList = create_model('ToolList', tools=(list[Union[tool1, tool2]], ...))
ToolList.model_json_schema()


{'$defs': {'Tool1': {'properties': {'name': {'const': 'tool1',
     'title': 'Name',
     'type': 'string'},
    'parameters': {'properties': {'operation': {'title': 'Operation',
       'type': 'string'},
      'a': {'title': 'A', 'type': 'number'},
      'b': {'title': 'B', 'type': 'number'}},
     'required': ['operation', 'a', 'b'],
     'title': 'calculateArguments',
     'type': 'object'}},
   'required': ['name', 'parameters'],
   'title': 'Tool1',
   'type': 'object'},
  'Tool2': {'properties': {'name': {'const': 'tool2',
     'title': 'Name',
     'type': 'string'},
    'parameters': {'properties': {},
     'title': 'get_current_timeArguments',
     'type': 'object'}},
   'required': ['name', 'parameters'],
   'title': 'Tool2',
   'type': 'object'}},
 'properties': {'tools': {'items': {'anyOf': [{'$ref': '#/$defs/Tool1'},
     {'$ref': '#/$defs/Tool2'}]},
   'title': 'Tools',
   'type': 'array'}},
 'required': ['tools'],
 'title': 'ToolList',
 'type': 'object'}

In [45]:
tools.tools[0].inputSchema

{'properties': {'operation': {'title': 'Operation', 'type': 'string'},
  'a': {'title': 'A', 'type': 'number'},
  'b': {'title': 'B', 'type': 'number'}},
 'required': ['operation', 'a', 'b'],
 'title': 'calculateArguments',
 'type': 'object'}

In [46]:
tools.tools[1].inputSchema

{'properties': {}, 'title': 'get_current_timeArguments', 'type': 'object'}

In [48]:
tool3 = create_model(
    'Tool3',
    name=(Literal['tool3'], ...),
    parameters=(object, ...)
    )
tool3.model_json_schema()

{'properties': {'name': {'const': 'tool3', 'title': 'Name', 'type': 'string'},
  'parameters': {'title': 'Parameters'}},
 'required': ['name', 'parameters'],
 'title': 'Tool3',
 'type': 'object'}

In [72]:

class CleanModel(BaseModel):
    @classmethod
    def remove_titles_recursively(cls,obj):
        if isinstance(obj, dict):
            if "title" in obj:
                del obj["title"]
            for value in obj.values():
                cls.remove_titles_recursively(value)
        elif isinstance(obj, list):
            for item in obj:
                cls.remove_titles_recursively(item)

    model_config = {
        "json_schema_extra": lambda schema, model: model.remove_titles_recursively(schema)
    }

# Tools definition (same as before)
tools = {
    "tools": [
        None,
        {
            "inputSchema": {
                "type": "object",
                "title": "InputParams",
                "properties": {
                    "x": {
                        "type": "string",
                        "title": "XValue",
                        "description": "Some X value"
                    }
                },
                "required": ["x"]
            }
        }
    ]
}

# Create the model
tool2 = create_model(
    'Tool2',
    __base__=CleanModel,
    name=(Literal['tool2'], ...),
    parameters=(object, Field(json_schema_extra=tools["tools"][1]["inputSchema"]))
)

tool2.model_json_schema()

{'properties': {'name': {'const': 'tool2', 'type': 'string'},
  'parameters': {'properties': {'x': {'description': 'Some X value',
     'type': 'string'}},
   'required': ['x'],
   'type': 'object'}},
 'required': ['name', 'parameters'],
 'type': 'object'}

In [80]:
tools.tools[0]

Tool(name='calculate', description="\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n", inputSchema={'properties': {'operation': {'title': 'Operation', 'type': 'string'}, 'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}, 'required': ['operation', 'a', 'b'], 'title': 'calculateArguments', 'type': 'object'})

In [85]:
tools.tools[0]['description']

TypeError: 'Tool' object is not subscriptable

In [90]:
print(tools.tools[0].description)


Perform a mathematical operation on two numbers.

Args:
    operation: One of 'add', 'subtract', 'multiply', 'divide'
    a: First number
    b: Second number
    
Returns:
    Result of the operation as a string



In [108]:
[tool.inputSchema for tool in tools.tools]

[{'properties': {'operation': {'title': 'Operation', 'type': 'string'},
   'a': {'title': 'A', 'type': 'number'},
   'b': {'title': 'B', 'type': 'number'}},
  'required': ['operation', 'a', 'b'],
  'title': 'calculateArguments',
  'type': 'object'},
 {'properties': {}, 'title': 'get_current_timeArguments', 'type': 'object'},
 {'properties': {'min_value': {'default': 1,
    'title': 'Min Value',
    'type': 'integer'},
   'max_value': {'default': 100, 'title': 'Max Value', 'type': 'integer'}},
  'title': 'generate_random_numberArguments',
  'type': 'object'}]

In [11]:
class CleanModel(BaseModel):
    @classmethod
    def remove_titles_recursively(cls,obj):
        if isinstance(obj, dict):
            if "title" in obj:
                del obj["title"]
            for value in obj.values():
                cls.remove_titles_recursively(value)
        elif isinstance(obj, list):
            for item in obj:
                cls.remove_titles_recursively(item)

    model_config = {
        "json_schema_extra": lambda schema, model: model.remove_titles_recursively(schema)
    }


def create_tool_model(mcp_name, tool):
    model_args = {}
    model_args['name'] = (Literal[mcp_name + '_' + tool.name], ...)
    if tool.inputSchema.get('properties'):
        model_args['parameters'] = (object, Field(json_schema_extra=tool.inputSchema))
    model_args['__base__'] = CleanModel
    model_args['model_config'] = {**CleanModel.model_config, "json_schema_extra": lambda schema, model: (
            CleanModel.model_config["json_schema_extra"](schema, model),
            schema.update({"description": tool.description})
        )}

    tool_model = create_model(
        tool.name,
        **model_args
    )
    return tool_model

def create_tool_model(tool):
    model_args = {}
    model_args['name'] = (Literal[tool.name], ...)
    if tool.inputSchema.get('properties'):
        model_args['parameters'] = (object, Field(json_schema_extra=tool.inputSchema))
    model_args['__doc__'] = tool.description
    model_args['__base__'] = CleanModel


    tool_model = create_model(
        '',
        **model_args
    )
    return tool_model



# m = create_model(
#     tools.tools[0].name,
#     __base__=CleanModel,
#     name=(Literal[tools.tools[0].name], ...),
#     parameters=(object, Field(json_schema_extra=tools.tools[0].inputSchema)),
#     model_config={
#         **CleanModel.model_config,
#         "json_schema_extra": lambda schema, model: (
#             CleanModel.model_config["json_schema_extra"](schema, model),
#             schema.update({"description": tools.tools[0].description})
#         )
#     }
#     )

In [13]:
tools_for_schema = [create_tool_model(tool) for tool in tools.tools]

In [15]:
from pydantic import RootModel
RootModel[list[Union[*tools_for_schema]]].model_json_schema()

{'$defs': {'__main______1': {'description': "Perform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string",
   'properties': {'name': {'const': 'calculate', 'type': 'string'},
    'parameters': {'properties': {'operation': {'type': 'string'},
      'a': {'type': 'number'},
      'b': {'type': 'number'}},
     'required': ['operation', 'a', 'b'],
     'type': 'object'}},
   'required': ['name', 'parameters'],
   'type': 'object'},
  '__main______2': {'description': 'Get the current date and time.',
   'properties': {'name': {'const': 'get_current_time', 'type': 'string'}},
   'required': ['name'],
   'type': 'object'},
  '__main______3': {'description': 'Generate a random number between min_value and max_value (inclusive).\n\nArgs:\n    min_value: Minimum value (default: 1)\n    max_value: Maximum value (default: 100)\n    \nRetu

In [134]:
create_tool_model('maths', tools.tools[0]).model_json_schema()

{'description': "Perform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string",
 'properties': {'name': {'const': 'maths_calculate',
   'default': 'maths_calculate',
   'type': 'string'},
  'parameters': {'properties': {'operation': {'type': 'string'},
    'a': {'type': 'number'},
    'b': {'type': 'number'}},
   'required': ['operation', 'a', 'b'],
   'type': 'object'}},
 'required': ['parameters'],
 'type': 'object'}

In [119]:
tools.model_dump()

{'meta': None,
 'nextCursor': None,
 'tools': [{'name': 'calculate',
   'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
   'inputSchema': {'properties': {'operation': {'title': 'Operation',
      'type': 'string'},
     'a': {'title': 'A', 'type': 'number'},
     'b': {'title': 'B', 'type': 'number'}},
    'required': ['operation', 'a', 'b'],
    'title': 'calculateArguments',
    'type': 'object'}},
  {'name': 'get_current_time',
   'description': 'Get the current date and time.',
   'inputSchema': {'properties': {},
    'title': 'get_current_timeArguments',
    'type': 'object'}},
  {'name': 'generate_random_number',
   'description': '\nGenerate a random number between min_value and max_value (inclusive).\n\nArgs:\n    min_value: Minimum value (default: 1)\n    max_value: Maximum value (defa

In [96]:
from typing import Optional

In [99]:
create_model('model', tools=(Optional[list[Union[m,m]]], ...)).model_json_schema()

{'$defs': {'calculate': {'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
   'properties': {'name': {'const': 'calculate', 'type': 'string'},
    'parameters': {'properties': {'operation': {'type': 'string'},
      'a': {'type': 'number'},
      'b': {'type': 'number'}},
     'required': ['operation', 'a', 'b'],
     'type': 'object'}},
   'required': ['name', 'parameters'],
   'type': 'object'}},
 'properties': {'tools': {'anyOf': [{'items': {'$ref': '#/$defs/calculate'},
     'type': 'array'},
    {'type': 'null'}],
   'title': 'Tools'}},
 'required': ['tools'],
 'title': 'model',
 'type': 'object'}

In [122]:
tools.tools[0].model_dump()

{'name': 'calculate',
 'description': "\nPerform a mathematical operation on two numbers.\n\nArgs:\n    operation: One of 'add', 'subtract', 'multiply', 'divide'\n    a: First number\n    b: Second number\n    \nReturns:\n    Result of the operation as a string\n",
 'inputSchema': {'properties': {'operation': {'title': 'Operation',
    'type': 'string'},
   'a': {'title': 'A', 'type': 'number'},
   'b': {'title': 'B', 'type': 'number'}},
  'required': ['operation', 'a', 'b'],
  'title': 'calculateArguments',
  'type': 'object'}}

# Make sure cleanup happens with the asyncexitstack -- explore further

In [27]:
import param

class A(param.Parameterized):
    a = param.Number(default=1)
    b = param.Number(default=2)

    @property
    def c(self):
        return_val = 2 + self.a
        self.a = return_val
        return  return_val


In [28]:
test_a = A()

In [43]:
test_a.c

31